# Random Forest

In [74]:
#importing required modules
import numpy as np
import math

In [75]:
def avg(target,indx):  #This function calcultes the avg of the data labels for given indices passed as argument
    if isinstance(indx,int):
        return target[indx]
    elif len(indx) == 0:
        return 0.0
    else:
        return sum([ target[i] for i in indx ]) / len(indx)

def rss(target,indx):     #This function calculates the rss for the given data at the indices passed as argument
    if len(indx) == 0 :
        return 0.0
    else:
        mean = avg(target,indx)
        return sum([ pow( target[i]-mean , 2.0 ) for i in indx ])

#Class to represent nodes of the decision tree
class decisionTree:
    #There are two types of nodes.
    #When the node is a leaf, then self.leaf = True, and the prediction is the average of the labels of the data reaching this leaf.
    #When the node is not a leaf, then self.attr and self.split record the optimal split, and self.L, self.R are two sub-decision-trees.

    def __init__(self,data,target,indx,depth):              #if you do not want to limit depth, you can set depth = len(data)
        if depth==0:										#if depth = 0, that means we don't go further down the tree
            self.leaf = True								#so it is a leaf
            self.prediction = avg(target,indx)				#and the prediction is the average of all labels in data[indx]
        elif isinstance(indx,int):                          #the case when len(indx)==1;
            self.leaf = True
            self.prediction = target[indx]
        elif len( set([target[i] for i in indx]) ) == 1:	#when all labels in data[indx] are the same, we don't need to go further down the tree
            self.leaf = True								
            self.prediction = target[indx[0]]				#and the prediction is simply the common label value
        else:												#otherwise, we need to do splitting; computing optimal split below
            self.leaf = False								#so it is not a leaf
            self.attr , self.split , self.L , self.R = self.generate(data,target,indx,depth)
    
    
    
    #generate is the function below, 
    #it randomly samples the attribute and chooses the best attribute for splitting and the optimal split using lowest rss method
    #attr stores which attribute is used to split
    #split stores the numerical value used to split into left and right subtrees
    #L and R are left and right subtrees

    def generate(self,data,target,indx,depth):						#generate the decision tree downwards
        attr =0
        split=0
        L=0
        R=0
        p = np.shape(data)[1]
        pn = math.ceil(p/3)
        p_indx = np.random.choice(range(p-1), pn, replace=False)
        opt = pow ( max(target) - min(target) , 2.0 ) * len(indx) + 1.0
        for j in p_indx:									#for each attribute, we search the optimal split 
            all_cuts = set( [ data[i][j] for i in indx ] )	#we find out all possible split values of the attribute we are considering
            for cut in all_cuts:
                yl = [ i for i in indx if data[i][j]<=cut ]	#yl is the list of indices to those observations where its j-th attribute value is <= cut
                yr = [ i for i in indx if data[i][j]>cut ]	#yr is the list of indices to those observations where its j-th attribute value is > cut
                tmp = rss(target,yl) + rss(target,yr)
                if tmp < opt:
                    opt , attr , split, L , R = tmp , j , cut , yl , yr
        return attr , split , decisionTree(data,target,L,depth-1) , decisionTree(data,target,R,depth-1)
        #after finding the optimal split, at each child node we recursively generate a decision tree of height depth-1

    #Function to traverse the decision tree and return the prediction for data passed as input
    def predict(self,x):
        if self.leaf == True:
            return self.prediction
        if (x[self.attr] <= self.split):
            return self.L.predict(x)
        else:
            return self.R.predict(x)     


In [76]:
#Function to generate sample BTS based on data and value of B passed as argument
#then it build the forest with B trees each of height h received as a argument
#stores the tree in a datastructure
#calculates training and test MSEs
#returns the test and training MSEs
def random_forest(X_train,y_train,X_test,y_test,B,h):
    n = len(X_train)
    trees = []	#this is used to store the decision trees generated
    test_mses = []
    b_test_predictions = []
    b_train_predictions = []
    for j in range(B):
        bts_indx = np.random.choice(range(n-1), n, replace=True)  #generating indices representing the BTS
        bts_X_train = X_train[bts_indx,:]                         #obtaining the training BTS
        bts_y_train = y_train[bts_indx]
        n = len(bts_X_train)
        all_indx = list(range(n))
        trees.append( decisionTree( bts_X_train,bts_y_train,all_indx , h ) )  #generating the decision tree with the BTS of height h
        #Making predictions on training and test sets using the tree
        test_predictions = []
        train_predictions = []
        for x in X_train:
            train_predictions.append(trees[-1].predict(x))
        b_train_predictions.append(train_predictions)    
        for x in X_test:
            test_predictions.append(trees[-1].predict(x))
        b_test_predictions.append(test_predictions)
    #Calculating final prediction averaged over all trees    
    avg_train_predictions = np.sum(b_train_predictions,axis=0)/len(b_train_predictions)
    train_mse = np.sum(np.square(avg_train_predictions - y_train))/len(y_train)  #Calculating training MSE
    #print("Training MSE:",train_mse)
    avg_test_predictions = np.sum(b_test_predictions,axis=0)/len(b_test_predictions)
    test_mse = np.sum(np.square(avg_test_predictions - y_test))/len(y_test)      #Calculating training MSE
    #print("Test MSE:",test_mse)
    return train_mse,test_mse

# Loading Dataset

In [77]:
from sklearn.datasets import load_boston
boston = load_boston()

In [78]:
print(boston.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

Splitting data into training and test sets

In [79]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(boston.data,boston.target,test_size=0.5,random_state=1214)

# Tests with B=100,h=3 on Boston Dataset

In [80]:
B=100
h=3
a,b = random_forest(X_train,y_train,X_test,y_test,B,h)
print("Training MSE:",a)
print("Training MSE:",b)

Training MSE: 14.628602392852027
Training MSE: 26.000549403370538


# MSE for Variations of B for fixed h=3

In [81]:
for B in [10,20,30, 40, 50, 60, 70, 80, 90, 100,200,300]:
    print("B =",B)
    a,b = random_forest(X_train,y_train,X_test,y_test,B,3)
    print("Training MSE:",a)
    print("Training MSE:",b)
    print()

B = 10
Training MSE: 15.527410410968264
Training MSE: 26.43852599084298

B = 20
Training MSE: 13.469917616170425
Training MSE: 27.096017176349363

B = 30
Training MSE: 14.471016661460727
Training MSE: 25.63843419802816

B = 40
Training MSE: 14.111998986541975
Training MSE: 26.032289824595743

B = 50
Training MSE: 14.737501220169168
Training MSE: 27.22014006335549

B = 60
Training MSE: 14.643941321008008
Training MSE: 27.04977854999381

B = 70
Training MSE: 15.162091004020512
Training MSE: 26.895218406842915

B = 80
Training MSE: 15.213757027075495
Training MSE: 27.372523411757772

B = 90
Training MSE: 13.827135834321137
Training MSE: 25.841843244966253

B = 100
Training MSE: 14.78678009828894
Training MSE: 27.134342963900853

B = 200
Training MSE: 14.09883989222337
Training MSE: 26.060700488746857

B = 300
Training MSE: 14.18834254583526
Training MSE: 26.624592439635688



# MSE for Variations of h for fixed B=100

In [64]:
for h in [1,2,3,4,5,6,7,8,9,10,11,12,13]:
    print("H =",h)
    a,b = random_forest(X_train,y_train,X_test,y_test,100,h)
    print("Training MSE:",a)
    print("Training MSE:",b)
    print()
    

H = 1
Training MSE: 38.879294448435274
Test MSE: 49.01026243854264
H = 2
Training MSE: 23.922147782493656
Test MSE: 33.19774121070799
H = 3
Training MSE: 13.85860023348743
Test MSE: 26.403744790292535
H = 4
Training MSE: 9.537953115689671
Test MSE: 23.594376286509178
H = 5
Training MSE: 6.694569235468292
Test MSE: 22.224319134908143
H = 6
Training MSE: 4.820174574095346
Test MSE: 21.220117666028557
H = 7
Training MSE: 4.125479727179251
Test MSE: 21.368806380000116
H = 8
Training MSE: 3.1584122917497486
Test MSE: 21.039654686594158
H = 9
Training MSE: 3.187229771790403
Test MSE: 20.35609327847366
H = 10
Training MSE: 2.9976867365729185
Test MSE: 20.565117053339787
H = 11
Training MSE: 2.424579261570438
Test MSE: 20.395300304244806
H = 12
Training MSE: 2.661363030052342
Test MSE: 20.727457510958335
H = 13
Training MSE: 2.47767838111036
Test MSE: 20.461821050158072


# MSE for variation of both B and H

In [84]:
m = np.zeros((156,4))
i =0
for B in [10,20,30, 40, 50, 60, 70, 80, 90, 100,200,300]: 
    for h in [1,2,3,4,5,6,7,8,9,10,11,12,13]:
        a,b = random_forest(X_train,y_train,X_test,y_test,B,h)
        m[i][0] = B
        m[i][1] = h
        m[i][2] = a
        m[i][3] = b
        i = i +1
print(m)        

[[ 10.           1.          37.34982985  45.37525661]
 [ 10.           2.          24.82580911  34.04497891]
 [ 10.           3.          18.53430832  29.13113254]
 [ 10.           4.          11.38531881  26.20632225]
 [ 10.           5.           9.79364936  23.45866607]
 [ 10.           6.           6.6028667   20.39310373]
 [ 10.           7.           4.63411719  21.72997654]
 [ 10.           8.           4.60628955  22.17271663]
 [ 10.           9.           4.07609457  24.29366293]
 [ 10.          10.           3.51228908  21.22959867]
 [ 10.          11.           3.94749604  23.25969118]
 [ 10.          12.           4.6943308   24.80746398]
 [ 10.          13.           4.30106667  23.53270455]
 [ 20.           1.          39.19902862  49.60239544]
 [ 20.           2.          25.91468447  37.19983829]
 [ 20.           3.          15.80606309  27.64398649]
 [ 20.           4.           9.86258361  24.34725485]
 [ 20.           5.           6.51233995  24.40697751]
 [ 20.    